In [7]:
import matplotlib
import torchvision
import torch
import torch.nn as nn
import numpy as np 
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
from einops import rearrange, repeat
%matplotlib inline
from thop import profile

def show_macs_params(net, a):
    macs, params = profile(net, inputs=(a,), verbose=False)  # macs == FLOPS, GFLOPS == 1e12 * FLOPS
    print(f"{net._get_name()}\t\t{macs=}\t{params=}")
    from thop import clever_format
    macs, params = clever_format([macs, params], "%.3f")
    print(f"{macs=}\t{params=}\n")

In [3]:
from models import MultiScaleAttentionHourglass
net = MultiScaleAttentionHourglass(4, 128, 24, [3, 5, 2])
x = torch.rand(1, 3, 256, 256)
show_macs_params(net, x)

h.shape=torch.Size([1, 128, 8, 8])
h.shape=torch.Size([1, 128, 16, 16])
h.shape=torch.Size([1, 128, 32, 32])
h.shape=torch.Size([1, 128, 64, 64])
MultiScaleAttentionHourglass		macs=1169090432.0	params=2312728.0
macs='1.169G'	params='2.313M'



In [10]:
from models import srhandnet
device = torch.device('cpu')
net = srhandnet(None).to(device)
x = torch.rand(1, 3, 256, 256)
# y = net(x)
show_macs_params(net, x)
# y[-1].shape

SRHandNet		macs=12144135680.0	params=18255032.0
macs='12.144G'	params='18.255M'



In [2]:
from models import litehrnet
# from models.lite_hrnet import LiteHRNet as Network
device = torch.device('cpu')

# state = torch.load('checkpoint/Center_SimDR/1HG-ME-att-c128-k2/73.82_AP_0epoch.pt', map_location=device)['state_dict']
# total_params = sum([v.nelement() for v in state.values()])
# print("Number of parameter: %.2fM" % (total_params / 1e6))

# for k, v in state.items():
#     print(f"{k}:\t {v.shape}")

a = torch.rand(1, 3, 256, 256)
net = litehrnet().to(device)
# 通过 thop 计算
show_macs_params(net, a)

/root/data/miniconda3/envs/TorchCV/lib/python3.8/site-packages/thop/vision/basic_hooks.py:92: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  kernel = torch.DoubleTensor([*(x[0].shape[2:])]) // torch.DoubleTensor(list((m.output_size,))).squeeze()


LiteHRNet		macs=559691660.0	params=1773484.0
macs='559.692M'	params='1.773M'



In [ ]:
size = 64
class SimDR(nn.Module):

    def __init__(self) :
        super(SimDR, self).__init__()
        
        self.pred_x = nn.Linear(size ** 2 , int(256 * 2)) 
        self.pred_y = nn.Linear(size ** 2, int(256 * 2))

    def forward(self, x):
        # kpts = self.vector_feature(feature)
        # kpts = rearrange(kpts, 'b c h w -> b c (h w)')
        b,c, h, w = x.shape
        x = x.view(b, c, -1)
        pred_x = self.pred_x(x)  # (b, c, w * k)
        pred_y = self.pred_y(x)  # (b, c, h * k)
        return pred_x, pred_y  

from thop import profile
from models.layers import LiteHG, LiteResidual, Residual, LiteBRC, InvertedResidual, GhostConv, DWConv, SplitDWConv, channel_attention3x3, GhostConv, ReducedConv1x1, ms_att, fuse_block
from models.LiteHourglass import BRC, StemBlock, LiteHourglass
from models.attention import NAM_Channel_Att
from models.hourglass_SA import ME_att, HourglassNet_SA, my_pelee_stem

print('successfully import package')

a = torch.rand(1, 21, size, size)
show_macs_params(SimDR(), a)

a = torch.rand(1, 3, 256, 256)
show_macs_params(StemBlock(64, 32), a)


size = 32
c = 128
a = torch.rand(1, c, size, size)

# net = NAM_Channel_Att(c)
# net = channel_attention3x3(c)
# net = InvertedResidual(c, c, 2)
show_macs_params(DWConv(c, c), a)
# net = GhostConv(c, c)
show_macs_params(SplitDWConv(c, c), a)
show_macs_params(channel_attention3x3(c), a)

# show_macs_params(LiteHG(4, c), a)
# net = Bi_HG(4, c)
# net = BRC(c, c)
# net = GhostConv(c, c)
# net = ReducedConv1x1(c, c)

show_macs_params(ME_att(c, c), a)
# net = ms_att(c)


# net = MS_HG(c, c * 4, 4, 4)
# net = LiteResidual(c, c)
# show_macs_params(net, a)

# net = LiteBRC(c, c)

# net = ME_att(c, c)

show_macs_params(BRC(c, c), a)
show_macs_params(Residual(c, c), a)
# # y = net(a)

print(f"完整网络的参数：")
a = torch.rand(1, 3, 128, 128)
# net = my_pelee_stem(128)
# net = LiteHourglass(1, 128)
net = HourglassNet_SA(1)
show_macs_params(net, a)

In [ ]:
class Stem(nn.Module):
    def __init__(self, inp_dim=128) :
        super().__init__() 
        self.pre = nn.Sequential(
            # Conv(3, 64, 7, 2, bn=True, relu=True),
            nn.Conv2d(3, 64, 7, 2, 3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            Residual(64, 128),
            nn.MaxPool2d(2, 2),
            Residual(128, 128),
            Residual(128, inp_dim)
        )
    def forward(self, x):
        return self.pre(x)

a = torch.rand(1, 3, 256, 256)
net = Stem()
show_macs_params(net, a)

In [ ]:
class channel_attention3x3(nn.Module):
    def __init__(self, channel=128):
        super().__init__()
        self.att = nn.Sequential(
                    nn.AdaptiveAvgPool2d((1, 1)),
                    nn.BatchNorm2d(channel),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(channel, channel, 1, 1, 0, groups=channel),      
                    # nn.Dropout(p=0.3),
                    nn.Flatten(),
                    # nn.Linear(channel, channel),
                    nn.Sigmoid(),  
                    # nn.BatchNorm1d(channel),
                    # nn.LeakyReLU(inplace=True),  
                    )
        

    def forward(self, x):
        att = self.att(x)
        b, c, _, _ = x.shape
        x = x * att.view(b, c, 1, 1)
        return x

a = torch.rand(1, 128, 64, 64)
net = channel_attention3x3()
show_macs_params(net, a)

In [1]:
from torch.nn import functional as F
a = torch.rand(1, 2, 5, 5)
# a = F.upsample_nearest(a, scale_factor=2)
a= F.interpolate(a, scale_factor=2, mode='nearest')
a.shape

NameError: name 'torch' is not defined

In [2]:
import torch
from torch import nn
softmax = nn.Softmax(dim=2)
LogSoftmax = nn.LogSoftmax(dim=2)  # [B,LOGITS]
criterion_ = nn.KLDivLoss(reduction='none')

hm_preds = torch.rand(2, 3, 4, 4)
hm_gts = torch.rand(2, 3, 4, 4)

b, c, h, w = hm_preds.shape
log_scores = LogSoftmax(hm_preds.view(b, c, -1))
gt_scorces = softmax(hm_gts.view(b, c, -1))
loss = torch.mean(criterion_(log_scores, gt_scorces), dim=2)
loss

tensor([[0.0045, 0.0062, 0.0032],
        [0.0041, 0.0055, 0.0034]])

In [6]:
a = torch.rand(2, 3)
a.mul(a)

tensor([[0.0424, 0.1075, 0.0152],
        [0.6122, 0.4424, 0.1948]])